<a href="https://colab.research.google.com/github/Aryamaan777/Accident-Prediction/blob/master/Stock_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
# Use 'colab' for working, but change to 'svg' or 'png' before saving to GitHub
pio.renderers.default = "png"

In [64]:
tickers = ['^GSPC', 'AAPL', 'TSLA', 'SPY']
data = yf.download(tickers, start="2019-01-03", end="2025-01-01", auto_adjust = False)

# Use Adjusted Close for price analysis
adj_close = data['Adj Close']
adj_close.head()

[*********************100%***********************]  4 of 4 completed


Ticker,AAPL,SPY,TSLA,^GSPC
Date,,,,
2019-01-03,33.799679,219.626282,20.024000,2447.889893
2019-01-04,35.242561,226.982834,21.179333,2531.939941
2019-01-07,35.164108,228.772537,22.330667,2549.689941
2019-01-08,35.834454,230.921936,22.356667,2574.409912
2019-01-09,36.442989,232.001114,22.568666,2584.959961


In [65]:
data.info

<bound method DataFrame.info of Price        Adj Close                                            Close  \
Ticker            AAPL         SPY        TSLA        ^GSPC        AAPL   
Date                                                                      
2019-01-03   33.799679  219.626282   20.024000  2447.889893   35.547501   
2019-01-04   35.242561  226.982834   21.179333  2531.939941   37.064999   
2019-01-07   35.164108  228.772537   22.330667  2549.689941   36.982498   
2019-01-08   35.834454  230.921936   22.356667  2574.409912   37.687500   
2019-01-09   36.442989  232.001114   22.568666  2584.959961   38.327499   
...                ...         ...         ...          ...         ...   
2024-12-24  257.037506  594.320740  462.279999  6040.040039  258.200012   
2024-12-26  257.853760  594.360413  454.130005  6037.589844  259.019989   
2024-12-27  254.439224  588.103821  431.660004  5970.839844  255.589996   
2024-12-30  251.064484  581.392578  417.410004  5906.939941  252.199997   
2024-12-31  249.292511  579.277405  403.839996  5881.629883  250.419998   

Price                                                  High              ...  \
Ticker             SPY        TSLA        ^GSPC        AAPL         SPY  ...   
Date                                                                     ...   
2019-01-03  244.210007   20.024000  2447.889893   36.430000  248.570007  ...   
2019-01-04  252.389999   21.179333  2531.939941   37.137501  253.110001  ...   
2019-01-07  254.380005   22.330667  2549.689941   37.207500  255.949997  ...   
2019-01-08  256.769989   22.356667  2574.409912   37.955002  257.309998  ...   
2019-01-09  257.970001   22.568666  2584.959961   38.632500  258.910004  ...   
...                ...         ...          ...         ...         ...  ...   
2024-12-24  601.299988  462.279999  6040.040039  258.209991  601.340027  ...   
2024-12-26  601.340027  454.130005  6037.589844  260.100006  602.479980  ...   
2024-12-27  595.010010  431.660004  5970.839844  258.700012  597.780029  ...   
2024-12-30  588.219971  417.410004  5906.939941  253.500000  591.739990  ...   
2024-12-31  586.080017  403.839996  5881.629883  253.279999  590.640015  ...   

Price              Low                     Open                          \
Ticker            TSLA        ^GSPC        AAPL         SPY        TSLA   
Date                                                                      
2019-01-03   19.825333  2443.959961   35.994999  248.229996   20.466667   
2019-01-04   20.181999  2474.330078   36.132500  247.589996   20.400000   
2019-01-07   21.183332  2524.560059   37.174999  252.690002   21.448000   
2019-01-08   21.801332  2547.560059   37.389999  256.820007   22.797333   
2019-01-09   22.098000  2568.889893   37.822498  257.559998   22.366667   
...                ...          ...         ...         ...         ...   
2024-12-24  435.140015  5981.439941  255.490005  596.059998  435.899994   
2024-12-26  451.019989  6007.370117  258.190002  599.500000  465.160004   
2024-12-27  426.500000  5932.950195  257.829987  597.539978  449.519989   
2024-12-30  415.750000  5869.160156  252.229996  587.890015  419.399994   
2024-12-31  402.540009  5868.859863  252.440002  589.909973  423.790009   

Price                       Volume                                    
Ticker            ^GSPC       AAPL        SPY       TSLA       ^GSPC  
Date                                                                  
2019-01-03  2491.919922  365248800  144140700  104478000  3858830000  
2019-01-04  2474.330078  234428400  142628800  110911500  4234140000  
2019-01-07  2535.610107  219111200  103139100  113268000  4133120000  
2019-01-08  2568.110107  164101200  102512600  105127500  4120060000  
2019-01-09  2580.000000  180396400   95006600   81493500  4088740000  
...                 ...        ...        ...        ...         ...  
2024-12-24  5984.629883   23234700   33160100   59551800  1757720000  
2024-12-26  6024.970215   27237100 

In [66]:
# Loop through each ticker in your DataFrame
for ticker in adj_close.columns:
    # 1. Calculate the Moving Averages
    sma50 = adj_close[ticker].rolling(window=50).mean()
    sma200 = adj_close[ticker].rolling(window=200).mean()

    # 2. Create a new figure for this specific ticker
    fig = go.Figure()

    # Add Adjusted Close Price
    fig.add_trace(go.Scatter(
        x=adj_close.index, y=adj_close[ticker],
        mode='lines', name=f'{ticker} Adj Close',
        line=dict(width=2, color='royalblue')
    ))

    # Add SMA 50 (Fast)
    fig.add_trace(go.Scatter(
        x=adj_close.index, y=sma50,
        mode='lines', name='50-day SMA',
        line=dict(width=1.5, dash='dash', color='orange')
    ))

    # Add SMA 200 (Slow)
    fig.add_trace(go.Scatter(
        x=adj_close.index, y=sma200,
        mode='lines', name='200-day SMA',
        line=dict(width=1.5, dash='dot', color='red')
    ))

    # 3. Format the chart
    fig.update_layout(
        title=f'{ticker} Price Trend & Moving Averages',
        xaxis_title='Date',
        yaxis_title='Price (USD)',
        template='plotly_white',
        hovermode='x unified', # Shows all values for a date on one tooltip
        legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01)
    )

    # 4. Show the graph
    fig.show()

In [67]:
# Fetch OHLC for a single ticker
ticker_ohlc = yf.download('^GSPC', start="2023-01-01")

# 2. FIX: Flatten the MultiIndex columns
if isinstance(ticker_ohlc.columns, pd.MultiIndex):
    ticker_ohlc.columns = ticker_ohlc.columns.get_level_values(0)

# Now your existing code will work!

fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    vertical_spacing=0.03, subplot_titles=('OHLC', 'Volume'),
                    row_width=[0.2, 0.7])

# Candlestick chart
fig.add_trace(go.Candlestick(x=ticker_ohlc.index,
                open=ticker_ohlc['Open'], high=ticker_ohlc['High'],
                low=ticker_ohlc['Low'], close=ticker_ohlc['Close'], name='Market Data'),
                row=1, col=1)

# Volume chart
fig.add_trace(go.Bar(x=ticker_ohlc.index, y=ticker_ohlc['Volume'], name='Volume'),
                row=2, col=1)

fig.update_layout(title='AAPL Candlestick and Volume Analysis', xaxis_rangeslider_visible=False)
fig.show()

/tmp/ipython-input-2248304825.py:2: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


In [68]:
# Calculate Daily Returns
returns = adj_close.pct_change().dropna()

# Histogram of Returns
fig = px.histogram(returns, x="AAPL", nbins=100, marginal="box",
                   title="Distribution of AAPL Daily Returns",
                   labels={'AAPL': 'Daily Return %'})
fig.show()

In [69]:
corr_matrix = returns.corr()

fig = px.imshow(corr_matrix, text_auto=True, aspect="auto",
                title="Asset Correlation Matrix",
                color_continuous_scale='RdBu_r') # Red for negative, Blue for positive
fig.show()

In [70]:
import numpy as np

# Annualize the metrics (252 trading days)
risk_return = pd.DataFrame({
    'Risk (Volatility)': returns.std() * np.sqrt(252),
    'Return': returns.mean() * 252
})

fig = px.scatter(risk_return, x='Risk (Volatility)', y='Return',
                 text=risk_return.index, title='Risk vs. Return Profile',
                 size_max=60)
fig.update_traces(textposition='top center')
fig.show()

In [71]:
cumulative_returns = (1 + returns).cumprod()

fig = px.line(cumulative_returns, title='Growth of $1 Investment (Cumulative Returns)',
              labels={'value': 'Portfolio Value', 'index': 'Date'})
fig.show()

In [72]:
# Calculate the running maximum
roll_max = adj_close.expanding().max()

# Calculate the drawdown as a percentage
drawdown = (adj_close - roll_max) / roll_max

# Visualize for all tickers
fig = px.line(drawdown,
              title="Drawdown Analysis (Percentage from Peak)",
              labels={'value': 'Drawdown %', 'Date': 'Date'})

# Add a horizontal line at 0 for reference
fig.add_hline(y=0, line_dash="dash", line_color="black")
fig.show()

In [73]:
import numpy as np

# 1. Calculate daily returns
returns = adj_close.pct_change()

# 2. Calculate 30-day rolling standard deviation and annualize it
# (Standard Deviation * Square root of 252 trading days)
rolling_vol = returns.rolling(window=30).std() * np.sqrt(252)

fig = px.line(rolling_vol,
              title="30-Day Rolling Annualized Volatility",
              labels={'value': 'Annualized Volatility (σ)', 'Date': 'Date'})
fig.show()

In [74]:
# We will look at a specific ticker, e.g., 'TSLA'
# target_ticker = 'TSLA'

for target_ticker in tickers:
  # Create a temporary dataframe for comparison
  vol_price_df = pd.DataFrame({
      'Daily Return': returns[target_ticker],
      'Volume': data['Volume'][target_ticker]
  }).dropna()

  fig = px.scatter(vol_price_df,
                  x='Volume',
                  y='Daily Return',
                  trendline="ols", # Adds a trend line to show correlation
                  title=f'Volume vs. Daily Return Relationship: {target_ticker}',
                  opacity=0.5)

  fig.update_layout(template='plotly_white')
  fig.show()

In [75]:
# 1. Calculate Daily Returns
returns = adj_close.pct_change().dropna()

# 2. Set the Risk-Free Rate (Assume 3% for current market conditions)
risk_free_rate = 0.03

# 3. Annualize the metrics (252 trading days in a year)
annualized_return = returns.mean() * 252
annualized_vol = returns.std() * np.sqrt(252)

# 4. Calculate Sharpe Ratio
sharpe_ratios = (annualized_return - risk_free_rate) / annualized_vol

# 5. Create a Comparison Table
sharpe_table = pd.DataFrame({
    'Annualized Return': annualized_return,
    'Annualized Volatility': annualized_vol,
    'Sharpe Ratio': sharpe_ratios
}).sort_values(by='Sharpe Ratio', ascending=False)

print(sharpe_table)

# 6. Visualize
import plotly.express as px
fig = px.bar(sharpe_table, x=sharpe_table.index, y='Sharpe Ratio',
             title='Risk-Adjusted Performance (Sharpe Ratio)',
             color='Sharpe Ratio', color_continuous_scale='RdYlGn')
fig.show()

        Annualized Return  Annualized Volatility  Sharpe Ratio
Ticker                                                        
AAPL             0.380786               0.305718      1.147419
TSLA             0.709601               0.644858      1.053877
SPY              0.181849               0.198173      0.766247
^GSPC            0.166853               0.201135      0.680404


In [76]:
# 1. Calculate Daily Returns
returns = adj_close.pct_change().dropna()

# 2. Choose the Benchmark
benchmark = '^GSPC'
tickers_to_compare = ['AAPL', 'TSLA', 'SPY']

# 3. Calculate Rolling Correlation (90-day window)
rolling_corr_df = pd.DataFrame(index=returns.index)

for ticker in tickers_to_compare:
    rolling_corr_df[ticker] = returns[ticker].rolling(window=90).corr(returns[benchmark])

# 4. Visualize the results
fig = px.line(rolling_corr_df,
              title="90-Day Rolling Correlation to S&P 500 (^GSPC)",
              labels={'value': 'Correlation Coefficient', 'Date': 'Date'},
              color_discrete_map={'SPY': 'black'}) # Highlight SPY in black for reference

# Add a reference line at 1.0 (Perfect Correlation) and 0.0 (No Correlation)
fig.add_hline(y=1.0, line_dash="dash", line_color="red", annotation_text="Perfect Correlation")
fig.add_hline(y=0.0, line_dash="dot", line_color="gray")

fig.update_layout(yaxis_range=[-0.2, 1.1], template='plotly_white')
fig.show()